In [121]:
import yfinance as yf
import numpy as np 
import pandas as pd
import matplotlib
import mplfinance as mpf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_finance import candlestick_ohlc
import scipy.stats as stats
from datetime import datetime, timedelta

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [122]:
""" Retrieve Data from yfinance
    :type ticker: string
    :type startDate: string '%Y-%m-%d'
    :type endDate: string '%Y-%m-%d'
    :rtype: pandas dataframe
    -----------------------
    yf.Ticker() -> takes ticker and returns tuple of ticker objects
    .history()  -> returns datafram corresponding to period specified
"""
def pullData(ticker, startDate, endDate):
    return yf.Ticker(ticker).history(start=startDate, end=endDate)

""" Generate a Candlestick Plot
    :type df: pandas dataframe
    :type mav: tuple
    :type ticker: string
"""

""" Return price at a given day
    :type date: str or pandas timestamp
    :type df: pandas dataframe
    :rtype: float
"""  
def getPrice(date, df):
    if df.loc[df.index==date].empty:
        print("ERR: Unable to getPrice(), No trading data for",date)
    return df.loc[df.index==date]['Open'][0]

""" Return all local minimums and maximums for a given timeframe
    :type df: pandas dataframe
    :type start: str or pandas timestamp
    :type end: str or pandas timestamp
    :rtype: dict{date:price}
"""  
def get_Peaks_and_Valleys(df, start, end):
    peaks = {}
    valleys = {}
    
    days = ( pd.to_datetime(end) -  pd.to_datetime(start)).days

    for i in range(0,days,3):    
        startDate = pd.to_datetime(start) + timedelta(i)

        while df.loc[df.index==startDate].empty:
            startDate = startDate + timedelta(1)

        initial_state = state(date=startDate, price=getPrice(startDate, df))

        climb = problem_hillclimb(initial=initial_state, df=df, objective_function=getPrice, stepsize=1)
        out = hill_climb(climb, n_iter=50)
        peaks[out.date] = out.price

        initial_state = state(date=startDate, price=getPrice(startDate, df))
        fall = problem_hillfall(initial=initial_state, df=df, objective_function=getPrice, stepsize=1)
        out = hill_fall(fall, n_iter=50)
        valleys[out.date] = out.price
        
    return peaks, valleys

"""~~~~~~~~~~~ FOR HILLCLIMBING ~~~~~~~~~~~"""

""" Define State of problem as a date and a time """ 
class state:
    def __init__(self, date, price):
        self.date = date
        self.price = price
    
class problem:
    """ Maintain everything that defines the problem
    :type initial_state: Class state
    :type current_state: Class state
    :type objective_function: defined function
    :type stepsize: int
    :type df: pandas dataframe
    """ 
    def __init__(self, initial, df, objective_function, stepsize):
        self.initial_state = initial
        self.current_state = initial
        self.objective_function = objective_function
        self.stepsize = stepsize
        self.df = df
    
    """ Return possible moves to take
    :rtype: array of dates
    """  
    def moves(self):
        all_moves = []
        
        newMoveForward = self.current_state.date + timedelta(self.stepsize)
        newMoveBackward = self.current_state.date - timedelta(self.stepsize)
        
        while df.loc[df.index==newMoveForward].empty:
            newMoveForward = newMoveForward + timedelta(self.stepsize)
        while df.loc[df.index==newMoveBackward].empty:
            newMoveBackward = newMoveBackward - timedelta(self.stepsize)
            
        all_moves.append(newMoveBackward)
        all_moves.append(newMoveForward)
        
        return all_moves
        
""" Performs hill climbing
    :type problem: Class problem
    :type n_iter: int
    :rtype: if peak found: Class state
            if no peak found: False
"""  
def hill_climb(problem, n_iter):
    # for t in some number of iterations:
    #     1. get a list of our available moves
    #     2. which move optimizes the objective function?
    #     3. do that move; update our status
    #     4. possible goal/convergence check
    
    for i in range(n_iter):
        nextMove, nextValue = problem.best_move()
        
        if nextValue <= problem.current_state.price: return problem.current_state
        
        problem.current_state.date, problem.current_state.price = nextMove, nextValue
        
    print("Reached n iterations.")
    return False

class problem_hillclimb(problem):
    """ Choose date that has higher price
    :rtype: datetime, int
    """  
    def best_move(self):
        all_moves = self.moves()
        
        obj_func = [self.objective_function(moves, self.df) for moves in all_moves]
        
        if obj_func[0] > obj_func[1]: return all_moves[0], obj_func[0]
        
        else: return all_moves[1], obj_func[1]
        
""" Performs hill falling
    :type problem: Class problem
    :type n_iter: int
    :rtype: if peak found: Class state
            if no peak found: False
"""          
def hill_fall(problem, n_iter):
    # for t in some number of iterations:
    #     1. get a list of our available moves
    #     2. which move optimizes the objective function?
    #     3. do that move; update our status
    #     4. possible goal/convergence check
    
    for i in range(n_iter):
        nextMove, nextValue = problem.best_move()
        
        if nextValue >= problem.current_state.price: return problem.current_state
    
        problem.current_state.date, problem.current_state.price = nextMove, nextValue
        
    print("Reached n iterations.")
    return False
   
class problem_hillfall(problem):
    """ Choose date that has lower price
    :rtype: datetime, int
    """  
    def best_move(self):
        all_moves = self.moves()
        
        obj_func = [self.objective_function(moves, self.df) for moves in all_moves]
        
        if obj_func[0] < obj_func[1]: return all_moves[0], obj_func[0]

        else: return all_moves[1], obj_func[1]


In [154]:
ticker = 'AAPL'
startDate = '2014-12-01'
endDate = '2016-02-01'

df = pullData(ticker, startDate, endDate)
peaks, valleys = get_Peaks_and_Valleys(df, start='2015-01-01', end='2016-01-01')



In [169]:
##FINDING SUPPORT



def supportFinder():
    ##initialize vector containing all the pertinent support data
    inita = np.array(list([]))
    initb = np.array(list([]))
    support = np.array(list(zip(inita,initb)))

    ##initialize the bucket we want to use in our search for support data
    bucket = .01

    date = np.array(list(valleys.keys())) ## get all the dates from our dictonary result
    price = np.array(list(valleys.values())) ##get all the prices from our dictionary result
    z = np.array(list(zip(date,price))) ##zip as a 2d array

    for x in range(len(z)):
        y = x
        for y in range(len(z)):
        
            if(z[x][0] not in support):
                if(z[y][0] not in support):
            
            
                    if (abs ((z[y][1]-z[x][1])/z[x][1]) < bucket): 
                        
                    
                        y = np.append(support, [z[x]]) 
                        support = y
    return support
                        


In [170]:
def resistanceFinder():
    

    ##initialize vector containing all the pertinent support data
    inita = np.array(list([]))
    initb = np.array(list([]))
    resistance = np.array(list(zip(inita,initb)))

    ##initialize the bucket we want to use in our search for support data
    bucket = .01

    date = np.array(list(peaks.keys())) ## get all the dates from our dictonary result
    price = np.array(list(peaks.values())) ##get all the prices from our dictionary result
    z = np.array(list(zip(date,price))) ##zip as a 2d array

    for x in range(len(z)):
    
        for y in range(len(z)):
        
            if(z[x][0] not in resistance):
                if(z[y][0] not in resistance):
            
            
                    if (abs ((z[y][1]-z[x][1])/z[x][1]) < bucket): 
                        
                    
                        y = np.append(resistance, [z[y]]) 
                        resistance = y
    
    return resistance



In [171]:
s = supportFinder()
print("SUPPORT VALUES")
print(s)


print("valleys")
print(valleys)

SUPPORT VALUES
[Timestamp('2015-01-06 00:00:00') 24.31 Timestamp('2015-01-14 00:00:00')
 24.88 Timestamp('2015-01-16 00:00:00') 24.42
 Timestamp('2015-01-27 00:00:00') 25.65 Timestamp('2015-02-02 00:00:00')
 26.93 Timestamp('2015-02-09 00:00:00') 27.15
 Timestamp('2015-02-26 00:00:00') 29.5 Timestamp('2015-03-03 00:00:00')
 29.54 Timestamp('2015-03-12 00:00:00') 28.01
 Timestamp('2015-03-16 00:00:00') 28.37 Timestamp('2015-03-23 00:00:00')
 29.11 Timestamp('2015-03-26 00:00:00') 28.12
 Timestamp('2015-03-30 00:00:00') 28.41 Timestamp('2015-04-01 00:00:00')
 28.59 Timestamp('2015-04-06 00:00:00') 28.51
 Timestamp('2015-04-09 00:00:00') 28.82 Timestamp('2015-04-17 00:00:00')
 28.75 Timestamp('2015-04-22 00:00:00') 29.08
 Timestamp('2015-05-01 00:00:00') 28.88 Timestamp('2015-05-07 00:00:00')
 28.7 Timestamp('2015-05-12 00:00:00') 28.89
 Timestamp('2015-05-18 00:00:00') 29.53 Timestamp('2015-05-20 00:00:00')
 29.9 Timestamp('2015-05-27 00:00:00') 29.98
 Timestamp('2015-06-02 00:00:00') 29

In [161]:
print("RESISTANCE VALUES")
r = resistanceFinder()
print(r)

RESISTANCE VALUES
[Timestamp('2014-12-29 00:00:00') 25.96 Timestamp('2015-01-09 00:00:00')
 25.7 Timestamp('2015-01-15 00:00:00') 25.09
 Timestamp('2015-01-26 00:00:00') 25.95 Timestamp('2015-01-28 00:00:00')
 26.83 Timestamp('2015-02-03 00:00:00') 27.03
 Timestamp('2015-02-06 00:00:00') 27.49 Timestamp('2015-02-24 00:00:00')
 30.45 Timestamp('2015-02-27 00:00:00') 29.77
 Timestamp('2015-03-04 00:00:00') 29.57 Timestamp('2015-03-19 00:00:00')
 29.49 Timestamp('2015-03-24 00:00:00') 29.14
 Timestamp('2015-03-31 00:00:00') 28.88 Timestamp('2015-04-07 00:00:00')
 29.23 Timestamp('2015-04-13 00:00:00') 29.4
 Timestamp('2015-04-21 00:00:00') 29.34 Timestamp('2015-04-28 00:00:00')
 30.8 Timestamp('2015-05-04 00:00:00') 29.66
 Timestamp('2015-05-11 00:00:00') 29.3 Timestamp('2015-05-15 00:00:00')
 29.68 Timestamp('2015-05-19 00:00:00') 30.06
 Timestamp('2015-05-26 00:00:00') 30.5 Timestamp('2015-05-28 00:00:00')
 30.33 Timestamp('2015-06-03 00:00:00') 30.05
 Timestamp('2015-06-11 00:00:00') 2